#OpenStreetMap Sample Project Data Wrangling with MongoDB

### <font color='blown'>Map Area: Taipei, Taiwan</font>

https://www.openstreetmap.org/export#map=11/25.0380/121.5634

### 0. Source Code

The source code I used to clean my raw data and insert into mongo db is writen in /src/clean_taipei_street_data.py.
Other files in the src directory are helpers that help me audit the raw map data and can be neglected.

### 1. Problems Encountered in the Map

#### 1.1 Too many types of tags without unified rules

There are 461 different types of tags in the whole data, if I make each of them as a hash key in my json, I will have lots of redundant keys that will make future investigation difficult. After looking the data, I figure out that there are lots of tags start with "addr:", so I can make the addr as first level key and put all those parts behind it as the second level key, like this:

    {
        'addr':{
            'city':'Taipei'
            'street':'xx Road.'
        }
    }
    
Moreover, there are three tags "name", "old_name" and "alt_name", and both of them have lots of different language version, I only need to keep the "name" in my json file and make those different version of language as the second level key in my json data.

After dealing with keys I listed above, there are only 171 distinct keys left and I keep them as it in my json file under the 'others' key

#### 1.2 There are both English and Chinese names in the 'is_in' tag

I noticed that the value in the 'is_in' tag have different languages in my data, most of them is written in Chinese, but some of them is written in English and the Chinese version is kept in the 'is_in:zh' tag. Some of the Chinese value also have 'is_in:en' tag to keep the English ones, but not all. So I finally decide to keep the Chinese version as the only value in my json file since most of them can find the Chinese name, but some of them can't find the English name. The only one value in 'is_in' tag that I can't find a Chinese name is 'Taipei City, Taiwan', but I know the Chinese name of it, so I just hardcode it into my file.

#### 1.3 Authenticity of the position recorded in the data

I want to make sure the street data I'm dealing with is in the right area as it should be, that is, the area of Taipei City. However, after doing some investigation, I realized that there are no official records that can tell me what's the exact range of one city. So I only check the following two things to give myself more confidence that the data is right.

1. make sure all the 'lon' and 'lat' data in all the nodes lie between the square box the OpenStreetMap website give me, that is, 121.2678~121.8590 and 24.8615~25.2143, and the result is positive.

2. make sure the most representative landscape in Taipei - "The Taipei 101" is in the right position. I compare the position data in the node with tag 'Taipei 101' with the data I got from wikipedia. Here is what I got:

    1. The data from OpenStreetData node : 25.0341934, 121.5636644
    2. The data from wikipedia : 25°2′1″N 121°33′52″E and can be converted into 25.0336111, 121.56444444
    
   There is a slight difference between this two positions, but the width of this building is about 642 meter (again, got from wikipedia) and that can be converted into 0.0058 degree, that means the different of the positions between the two data is smaller than the width of the building itself. I think the measurement error is under tolerance.

### 2. Data Overview

1. File size
     > taipei_city_taiwan.osm : 70MB

2. All types of tags and their number of occurences in data

     > 'nd': 357285
     
     > 'node': 298526
     
     > 'tag': 235966
     
     > 'member': 54052
     
     > 'way': 50332
     
     > 'relation': 3122
     
     > 'osm': 1
     
     > 'bounds': 1

3. Top ten frequently used tags except for 'addr' and 'name' related tags

     > 'public_transport:version : 718
     
     > 'building:levels' : 469
     
     > 'GNS:id' : 295
     
     > 'GNS:dsg_string' : 295
     
     > 'GNS:dsg_code' : 295
     
     > 'diet:vegan' : 85
     
     > 'diet:vegetarian' : 78
     
     > 'payment:bitcoin' : 58
     
     > 'traffic_signals:sound' : 56
     
     > 'payment:coins' : 51
  
4. Number of documents
                                                
    > db.char.find().count()
      
    > 351980

5. Number of nodes, ways and relations                                                
    > - db.taipei_street.find({"type":"node"}).count(): 
    
    > 298526
    
    > - db.taipei_street.find({"type":"way"}).count():
    
    > 50332
    
    > - db.taipei_street.find({"type":"relation"}).count():
    
    > 3122
                                                
6. Number of unique users
    > db.taipei_street.distinct("created.user").length
    
    > 956
    
7. Plot and accumulation of the number of contributions of the top 30 contributing users

    > db.taipei_street.aggregate([{"\$group":{"_id":"\$created.user", "count":{"\$sum":1}}}, {"\$sort":{'count':-1}}, {"\$limit":30}])
    
    ![](http://i.imgur.com/CHmDYP6.png)
    
    > db.taipei_street.aggregate([{"\$group":{"_id":"\$created.user", "count":{"\$sum":1}}}, {"\$sort":{'count':-1}}, {"\$limit":30}, {"\$group":{"_id":null, sum: {"\$sum":'\$count'}}}])
    
    > 287436
    
    > This shows that most of the contributions is made by the top 30 users (81%), and the top one user gives us about 1/3 of the data. 

    

### 3. Additional Ideas

#### 3.1 Cross validation location information

1. What I did:

    - Using google API to find the location of the addresses provided by each node and compare it with the 'pos' provide by the node itself. If the distance of the two point is less than 500 meter, I will consider it as a tolerable measurement error, otherwise, I think it may need some more investigations to check the correctness of the position or address information provided by the node. 
<br/>
<br/>
2. Benifit:

    - If a node in OpenStreetData has address information, it mean this node is a standalone point features instead of a dull point used by way and relation only. Using google map API can double check if the location and address provided by this node is correct.
<br/>
<br/>
3. Prep work:
<br/>
<br/>
    1. There are 1066 nodes with 'address.full' information in the all 298526 nodes. I pick them out by:
        > db.taipei_street.find({"type":'node', "address.full":{"$exists":1}},{"_id":0, "pos":1, "address.full":1})
    
    2. I don't need the 'floor' information for the location of the nodes, so I remove them to make the google api find these locations easier.
    
    3. If there are more than one location find by google api, I will pick the first one to use.
<br/>
<br/>    
4. Conclusion:

    1. For all the nodes, there are no locations come from the two data sources that are exactly the same. This may clear the doubt that some of the position informations is just copied from the google map api in the first place.
    
    2. There are 11 addresses have more than one location informations from google map api, but using the first location doesn't get any noticeable problem. 
    
    3. Only one node have wrong address format and can't find any location information from google map api.
    
    4. 11 nodes have right address format but can't find any location information from google map api, these addresses may need furthur investigation in the future.
    
    5. 3 nodes have more than 500 meters but less than 1 kilometer deviation from the two sources of locations
    
    6. 3 nodes have more than 1 kilometer deviation from the two sources of locations (one of them have 6.368 kilometer deviation, this one may need some furthur investigation to check the correctness)
    
    7. Other 1037 nodes have right position with tolerable measurement error after cross validating with google map api. It means 97% of the nodes which have address information provide creditable location information.
<br/>
<br/>    
5. Further Discussion: 

    1. One of the problem is that if the two data soucres doesn't provide the same location informaiton, I don't know which one is right, I may need to find a third party resource to check or just decide by which one I trust more.
    
    2. It may be helpful if the OpenStreetData website can auto check with the google map api once a new node is created, it may tell the provider if this node's location or address information need some furthur investigation.
    
    3. I noticed that the Google Map Api doesn't allow me to do too much inquiries in the same time. It may be some insurance mechanism of Google to prevent from DDoS attack. If the OpenStreedData want to auto check with the Google Map Api, it may need some business cooperation with the Google Map Team to make sure it will give them the right to inquiry as many information as needed.
<br/>
<br/> 
6. Source code:
    - related code is written in /src/cross_validating_node_position_by_google_map_api.py

#### 3.2 Additional data exploration

1. The max version:
    to find the max version, I need to convert the value of version from integer to int in the first place.
    
    > for x in db.taipei_street.find():
    
        >> x['created']['version'] = int(x['created']['version']);
      
        >> db.taipei_street.save(x);
    
    then I can find the max version by sort:
        
    > db.taipei_street.aggregate([{"\$group":{"_id":"\$created.version", "count":{"\$sum":1}}},{"\$sort":{"_id":-1}},{"\$limit":1}])
    
    > 168
       
2. Top 10 appearing buildings:

    > db.taipei_street.aggregate([{"\$match":{"others.building":{"\$exists":1}}},{"\$group":{"_id":"\$others.building","count":{"\$sum":1}}}, {"\$sort":{"count":-1}}, {"\$limit":10}])

    the top one is "yes" and I don't know what it means, I think I should have removed this one from the begininng of data cleaning, but I don't know how to construct a solid rule to remove all unresonable buildings. Beside this, the top 10 appearing buildings are as below:
    
    > { "_id" : "commercial", "count" : 741 }
    
    > { "_id" : "school", "count" : 519 }
    
    > { "_id" : "residential", "count" : 405 }
    
    > { "_id" : "apartments", "count" : 371 }
    
    > { "_id" : "university", "count" : 366 }
    
    > { "_id" : "office", "count" : 220 }
    
    > { "_id" : "hospital", "count" : 116 }
    
    > { "_id" : "dormitory", "count" : 97 }
    
    > { "_id" : "industrial", "count" : 94 }
    
    > { "_id" : "house", "count" : 76 }

3. Top 5 appearing naturals:

    Since there are a lot of mountains in Taipei, it's not surprising that the most appearing naturl is "tree" and three of the top five naturals are related with tree.

    db.taipei_street.aggregate([{"\$match":{"others.natural":{"\$exists":1}}},{"\$group":{"_id":"\$others.natural", "count":{"\$sum":1}}}, {"\$sort":{"count":-1}}, {"\$limit":5}])
    
    > { "_id" : "tree", "count" : 1482 }
    
    > { "_id" : "wood", "count" : 169 }
    
    > { "_id" : "water", "count" : 159 }
    
    > { "_id" : "peak", "count" : 145 }
    
    > { "_id" : "tree_row", "count" : 87 }

4. When I looked at the top 10 occurrences tags, one of the tag 'payment:bitcoin' drew my attention. Did it means that these places can using bitcoin to pay the bill? So I try to find all the places that have value 'yes' for this key and find out that there are 57 places fit this criteria!

    > db.taipei_street.find({"others.payment:bitcoin":{"\$eq":'yes'}},{'_id':0, 'others.payment:bitcoin':1, 'name':1}).count()

    > 57

    Then I try to google some of these names and find out this : https://twitter.com/SoloSingerInn/status/425967541291192320/photo/1 

    > "Solo Singer: We are the first (and only) hotel in Taiwan to accept bitcoin, please help us promote it!"

    It's amazing since I never know there is a hotel that accept bitcoin in Taiwan before. The OpenStreetMap data just told me something I don't expect to see before auditing them. Maybe I should find a day to visit this special hotel in my hometown :)